In [1]:
import sys, os, json
import numpy as np
import pandas as pd
import seaborn as sns
import argparse, progressbar
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import matplotlib.axes._axes as axes
sns.set()

In [2]:
v = np.array([2, -1])

In [4]:
v.shape

(2,)

In [5]:
v3 = np.array([
    [2],
    [5],
    [4],
    [7]
])

In [6]:
v3.shape

(4, 1)

In [7]:
v4 = v3.T

In [8]:
v4.shape

(1, 4)

In [9]:
v4

array([[2, 5, 4, 7]])